# Tarea 3: Piping y Bombas
### Pablo Correa e Ian Gross

## Librerías

In [1]:
import CoolProp.CoolProp as cp
import fluids as fld
import numpy as np
import scipy.constants as cte
import scipy.optimize as opt
import matplotlib.pyplot as plt
import pandas as pd

from fluids.units import *

In [2]:
g = cte.g*u.m/u.s**2
Patm = (1*u.atm).to(u.Pa)

# Pregunta 1
Considere el sistema de almacenamiento de energía visto en el laboratorio.

A partir de los datos de operación del archivo anexo, verifique si la pérdida de carga observada se puede representar por la ecuación de Ergun.

¿Cuánto es el error con respecto a la correlación? (3pts), ¿Cuanto afecta la temperatura y la variabilidad de las propiedades termofísicas del aire? (3pts).

Asuma que el sistema de almacenamiento es un intercambiador de calor de flujo paralelo, donde el solido es uno de los medio de transferencia de calor. ¿Cuanto sería la efectividad del sistema (intercambiador de calor)? (3pts)

In [3]:
headers = ["Scan Sweep Time (s)", "Temp. Entrada (°C)", "Vel. Aire (m/s)", "Temp. Aire (°C)","Presión Dif. Estanque (mbar)", "Temp. Salida (°C)"]
data = pd.read_excel("Ensayo Piloto Lleno T-220°C.xlsx", skiprows=16, usecols="A,C:G", names=headers)
data

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
data.corr()

,Temp. Entrada (°C),Vel. Aire (m/s),Temp. Aire (°C),Presión Dif. Estanque (mbar),Temp. Salida (°C)
Temp. Entrada (°C),1.000000,0.586682,0.779613,0.665450,0.745914
Vel. Aire (m/s),0.586682,1.000000,0.815704,0.820089,0.520827
Temp. Aire (°C),0.779613,0.815704,1.000000,0.963905,0.798176
Presión Dif. Estanque (mbar),0.665450,0.820089,0.963905,1.000000,0.736789
Temp. Salida (°C),0.745914,0.520827,0.798176,0.736789,1.000000


# Pregunta 2
Liquid  carbon  dioxide  enters  the  tubes  of  a  shell  and  tube  heat  exchanger  at  a  temperature of 0°C and a flow rate of 30 kg/s. The carbon dioxide is heated with a flow of city water that enters the shell side of the heat exchanger at a temperature of 40°C and a flow rate of 31 kg/s. The heat exchanger is a single shell, two-tube pass configuration with a 25in shell. The tubes are ¾in 10BWG laid out on a 1in triangular pitch. The heat exchanger is 2m long and contains 2 baffles per meter of length.
* Asumiendo presión atmosférica en la entrada
* Asumiendo que las paredes del intercambiador son de hierro forjado. [1]
* Asumiendo que el cambio de $\mu$ es despreciable entre la coraza y la pared del tubo

In [95]:
# Temperaturas
T_CO2 = (0*u.degC).to(u.K)
T_CityWater = (40*u.degC).to(u.K)

# Masas
dm_CO2 = 30*u.kg/u.s
dm_CityWater = 31*u.kg/u.s

# Diametros
d_shell = (25*u.inch).to(u.m)
do_tube = (0.75*u.inch).to(u.m)
t_tube = t_from_gauge(10, schedule='BWG')
di_tube = do_tube - t_tube

# Longitudes
l = 2*u.m
Pt = (1*u.inch).to(u.m)
B = 0.5*u.m

# Número de tubos
N_T = 452*u.dimensionless

# Número de pasos
N_P = 2*u.dimensionless

# Número de bafles
N_B = 4*u.dimensionless

# Fouling Factor
R_CO2 = 0.000176*u.m**2*u.K/u.W
R_CityWater = 0.000352*u.m**2*u.K/u.W

In [71]:
# Diámetro hidáulico dado en clases
d_shell_eq = 2*np.sqrt(3)*Pt**2/(np.pi*do_tube)-do_tube
print(f"d_shell_eq = {d_shell_eq:.4f}")

# Área efectiva de transferencia de calor
A_tube_eff = N_T/N_P*(np.pi*di_tube**2/4)
print(f"A_tube_eff = {A_tube_eff:.4f}")

# Área transversal de un tubo
A_tube_cross = np.pi * di_tube**2/4*N_T

# Area efectiva de transferencia de calor
A_shell_eff = (d_shell*(Pt-do_tube)*B/Pt).to(u.m**2)
print(f"A_shell_eff = {A_shell_eff:.4f}")



d_shell_eq = 0.0183 meter
A_tube_eff = 0.0435 meter ** 2
A_shell_eff = 0.0794 meter ** 2


In [80]:
# Propiedades de los fluidos
rho_CO2 = cp.PropsSI('D', 'T', T_CO2.magnitude, 'Q', 0, 'CO2')*u.kg/u.m**3
mu_CO2 = cp.PropsSI('V', 'T', T_CO2.magnitude, 'Q', 0, 'CO2')*u.Pa*u.s
k_CO2 = cp.PropsSI('L', 'T', T_CO2.magnitude, 'Q', 0, 'CO2')*u.W/u.m/u.K
cp_CO2 = cp.PropsSI('Cpmass', 'T', T_CO2.magnitude, 'Q', 0, 'CO2')*u.J/u.kg/u.K


rho_CityWater = cp.PropsSI('D', 'T', T_CityWater.magnitude, 'P', Patm.magnitude, 'water')*u.kg/u.m**3
mu_CityWater = cp.PropsSI('V', 'T', T_CityWater.magnitude, 'P', Patm.magnitude, 'water')*u.Pa*u.s
k_CityWater = cp.PropsSI('L', 'T', T_CityWater.magnitude, 'P', Patm.magnitude, 'water')*u.W/u.m/u.K
cp_CityWater = cp.PropsSI('Cpmass', 'T', T_CityWater.magnitude, 'P', Patm.magnitude, 'water')*u.J/u.kg/u.K

# Cp
C_CO2 = dm_CO2*cp_CO2
C_CityWater = dm_CityWater*cp_CityWater
Cmin = min(C_CO2, C_CityWater)
Cmax = max(C_CO2, C_CityWater)
Cr = Cmin/Cmax
print(f"Cr = {Cr:.2f}")

Cr = 0.59 dimensionless


### Calculando h_interno

In [73]:
# Condiciones para Nusselt
V_CO2 = dm_CO2/(rho_CO2*A_tube_eff)

Re_CO2 = fld.core.Reynolds(V_CO2, di_tube, rho_CO2, mu_CO2).to(u.dimensionless)
print(f"Re_CO2 = {Re_CO2:.2f}")
Pr_CO2 = fld.core.Prandtl(Cp=cp_CO2, mu=mu_CO2, k=k_CO2).to(u.dimensionless)
print(f"Pr_CO2 = {Pr_CO2:.2f}")

print(f"L/D = {l/di_tube:.2f}")

Re_CO2 = 107590.86 dimensionless
Pr_CO2 = 2.34 dimensionless
L/D = 127.83 dimensionless


Para calcular el número de Nusselt dado que el flujo es turbulento y se está calentando:

In [74]:
Nu_CO2 = 0.023*Re_CO2**0.8*Pr_CO2**0.4
print(f"Nu_CO2 = {Nu_CO2:.2f}")

h_tube = (k_CO2*Nu_CO2/t_tube).to(u.W/u.m**2/u.K)
print(f"h_tube = {h_tube:.2f}\n")


Nu_CO2 = 342.55 dimensionless
h_tube = 10983.97 watt / kelvin / meter ** 2



### Calculando R de conducción

In [88]:
k = 45*u.W/u.K/u.m
Rw = np.log(do_tube/di_tube)/(2*np.pi*k*l)/N_T
print(f"Rw = {Rw:.5}\n")

Rw = 7.7016e-07 kelvin / watt



### Calculando h_externo

In [89]:
V_shell = dm_CityWater/(rho_CityWater*A_shell_eff)
print(f"Vs = {V_shell:.3f}")

Re_CityWater = fld.core.Reynolds(V_shell, d_shell_eq, rho_CityWater, mu_CityWater).to(u.dimensionless)
print(f"Re_CityWater = {Re_CityWater:.2f}")

Pr_CityWater = fld.core.Prandtl(Cp=cp_CityWater, mu=mu_CityWater, k=k_CityWater).to(u.dimensionless)
print(f"Pr_CityWater = {Pr_CityWater:.2f}")

Vs = 0.394 meter / second
Re_CityWater = 10945.57 dimensionless
Pr_CityWater = 4.34 dimensionless


Para calcular el número de Nusselt dado que el flujo es turbulento y se está enfriando:

In [90]:
Nu_CityWater = 0.36*Re_CityWater**0.55*Pr_CityWater**0.33
print(f"Nu_CityWater = {Nu_CityWater:.2f}")

h_shell = (k_CityWater*Nu_CityWater/d_shell_eq).to(u.W/u.m**2/u.K)
print(f"h_shell = {h_shell:.2f}\n")

Nu_CityWater = 97.34 dimensionless
h_shell = 3344.09 watt / kelvin / meter ** 2



### Cálculo de U

In [91]:
UA = ((1/(h_tube*A_tube_eff) + Rw + 1/(h_shell*A_shell_eff))**-1).to(u.W/u.K)
print(f"UA = {UA:.2f}")

UA = 170.55 watt / kelvin


### Cálculo de NTU

In [94]:
print(f"Cr = {Cr:.2f}")
NTU = (UA/Cmin).to(u.dimensionless)
print(f"NTU = {NTU}\n")
e = 2*(1+Cr+np.sqrt(1+Cr**2)*(1+np.exp(-NTU*np.sqrt(1+Cr**2)))/(1-np.exp(-NTU*np.sqrt(1+Cr**2))))**-1
print(f"Efficiency = {e}")

Cr = 0.59 dimensionless
NTU = 0.002236153515557553 dimensionless

Efficiency = 0.0022321873267121366 dimensionless


### LMTD y correción

In [83]:
T_CO2_out = (20*u.degC).to(u.K)
T_CityWater_out = (20*u.degC).to(u.K)

R = C_CO2/C_CityWater
P = (T_CO2_out-T_CO2)/(T_CityWater-T_CO2)
F = (np.sqrt(R**2+1)*np.log((1-P)/(1-P*R)))/((R-1)*np.log((2-P*(R+1-np.sqrt(R**2+1)))/(2-P*(R+1+np.sqrt(R**2+1))))).to(u.dimensionless)
print(f"F = {F:.2f}")

F = 0.93 dimensionless


In [ ]:
Qmax = Cmin*(T_CityWater-T_CO2)

# Single shell double tube heat exchanger

a. The outlet temperatures and pressure drops of each fluid in the heat exchanger for the case of a brand new heat exchanger just put into service.

In [ ]:
f_CO2 = fld.core.friction_factor(Re_CO2, eD=di_tube, roughness=0*u.m)

b. The outlet temperatures of each fluid after the heat exchanger has been in continuous service for one year without maintenancece.

c. If the outlet temperature of the carbon dioxide must be 20°C or higher, is this heat exchanger sufficient for the required duty?

# Pregunta 3
Considere el plano hidráulico de la localidad de Hualañé y la distribución de diámetros determinada en la Tarea 2. Seleccione una bomba de uno de los dos catalogos disponibles, que permita entregar el caudal y nivel de presión requerido (6pts). Identifique el punto de operación de la bomba (6pts)

# Bibliografía

[1] https://www.stainless-structurals.com/blog/comparing-the-thermal-conductivity-of-stainless-steel-to-other-metals/